In [9]:
from collections import Counter
import numpy as np

corpus = "low low low low low lower lower widest widest widest newest newest newest newest newest newest"


def pretokenize(text):
    result = text.split(" ")
    freq_table = dict(Counter(result))
    # freq_table = {tuple(list(k.encode('utf-8'))): v for k, v in freq_table.items()}
    freq_table = {tuple(list(k)): v for k, v in freq_table.items()}
    return freq_table


def count_pairs(freq_table):
    count = Counter()
    for k, v in freq_table.items():
        if len(k) > 1:
            for i in range(len(k) - 1):
                count[k[i] + k[i + 1]] += v
    return dict(count)


def merge(freq_table, count):
    keys = np.array(list(count))
    values = np.array(list(count.values()))
    indexes = np.where(values == values.max())[0]
    max_keys = keys[indexes]
    index = np.argmax(max_keys)
    merge_key = max_keys[index]
    new_freq_table = {}
    for key in list(freq_table):
        new_key = []
        skip_next = False
        for i in range(len(key)):
            if skip_next:
                skip_next = False
                continue
            if i == len(key) - 1:
                new_key.append(key[i])
                break
            pair = key[i] + key[i + 1]
            if pair == merge_key:
                new_key.append(pair)
                skip_next = True
            else:
                new_key.append(key[i])
                skip_next = False
        new_freq_table[tuple(new_key)] = freq_table[key]
    return new_freq_table, merge_key


def one_round(freq_table):
    count = count_pairs(freq_table)
    new_freq_table, merge_key = merge(freq_table, count)
    print("count:", count, merge_key)
    print("old ft:", freq_table)
    print("new ft:", new_freq_table)
    return new_freq_table


freq_table = pretokenize(corpus)
for i in range(12):
    if i != 0:
        print("")
    print(f"round {i+1}")
    freq_table = one_round(freq_table)

round 1
count: {'lo': 7, 'ow': 7, 'we': 8, 'er': 2, 'wi': 3, 'id': 3, 'de': 3, 'es': 9, 'st': 9, 'ne': 6, 'ew': 6} st
old ft: {('l', 'o', 'w'): 5, ('l', 'o', 'w', 'e', 'r'): 2, ('w', 'i', 'd', 'e', 's', 't'): 3, ('n', 'e', 'w', 'e', 's', 't'): 6}
new ft: {('l', 'o', 'w'): 5, ('l', 'o', 'w', 'e', 'r'): 2, ('w', 'i', 'd', 'e', 'st'): 3, ('n', 'e', 'w', 'e', 'st'): 6}

round 2
count: {'lo': 7, 'ow': 7, 'we': 8, 'er': 2, 'wi': 3, 'id': 3, 'de': 3, 'est': 9, 'ne': 6, 'ew': 6} est
old ft: {('l', 'o', 'w'): 5, ('l', 'o', 'w', 'e', 'r'): 2, ('w', 'i', 'd', 'e', 'st'): 3, ('n', 'e', 'w', 'e', 'st'): 6}
new ft: {('l', 'o', 'w'): 5, ('l', 'o', 'w', 'e', 'r'): 2, ('w', 'i', 'd', 'est'): 3, ('n', 'e', 'w', 'est'): 6}

round 3
count: {'lo': 7, 'ow': 7, 'we': 2, 'er': 2, 'wi': 3, 'id': 3, 'dest': 3, 'ne': 6, 'ew': 6, 'west': 6} ow
old ft: {('l', 'o', 'w'): 5, ('l', 'o', 'w', 'e', 'r'): 2, ('w', 'i', 'd', 'est'): 3, ('n', 'e', 'w', 'est'): 6}
new ft: {('l', 'ow'): 5, ('l', 'ow', 'e', 'r'): 2, ('w', 'i